___

<a href='https://www.udemy.com/user/joseportilla/'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Content Copyright by Pierian Data</em></center>

# Milestone Project 2 - Blackjack Game
In this milestone project you will be creating a Complete BlackJack Card Game in Python.

Here are the requirements:

* You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
* The game needs to have one player versus an automated dealer.
* The player can stand or hit.
* The player must be able to pick their betting amount.
* You need to keep track of the player's total money.
* You need to alert the player of wins, losses, or busts, etc...

And most importantly:

* **You must use OOP and classes in some portion of your game. You can not just use functions in your game. Use classes to help you define the Deck and the Player's hand. There are many right ways to do this, so explore it well!**


Feel free to expand this game. Try including multiple players. Try adding in Double-Down and card splits! Remember to you are free to use any resources you want and as always:

# HAVE FUN!

In [1]:
# IMPORTS
import random
import time
from IPython.display import clear_output

# GLOBAL VARIABLES
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace': (1,11)}

In [2]:
class Card:
    
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
    
    def __str__(self):
        return f"{self.rank} of {self.suit}"

In [3]:
class Deck:
    
    def __init__(self):
        self.cards = []
        
        for suit in suits:
            for rank in ranks:
                new_card = Card(suit, rank)
                self.cards.append(new_card)
    
    def shuffle(self):
        random.shuffle(self.cards)
    
    def deal_one(self):
        return self.cards.pop()
    
    def __str__(self):
        return f"There are {len(self.cards)} cards in the deck"

In [30]:
class Player:
    
    def __init__(self):
        self.balance = 100
        self.total = 0
        self.cards = []
    
    def add_cards(self,card):
        self.cards.append(card)
        
        if card.rank == "Ace":
            eleven = self.total + 11
            one = self.total + 1
            if eleven == 21:
                self.total += 11
            elif one == 21:
                self.total += 1
            elif eleven < 21:
                self.total += 11
            else:
                self.total += 1
        else:
            self.total += card.value
        
    def collect_bet(self, bet):
        self.balance = self.balance + bet
        
    def remove_hand(self):
        self.cards = []
        self.total = 0
        
    def place_bet(self):
        invalid_bet = True
        
        while invalid_bet:            
            try:
                val = int(input("How much would you like to bet? "))
            except:
                print("The amount you entered is not valid.")
            else:
                if val > self.balance:
                    print("Sorry, you don't that much to bet.")
                else:
                    self.balance = self.balance - val
                    print(f"Your {val} bet has been placed")
                    invalid_bet = False
        
        return val

In [31]:
class Dealer(Player):
    
    def __init__(self):
        Player.__init__(self)
        self.face_down_card = []
    
    def add_face_down_card(self, card):
        self.face_down_card.append(card)
        
    def flip_card(self):
        flipped_card = self.face_down_card.pop()
        self.add_cards(flipped_card)

In [32]:
'''
FUNCTIONS
'''

# Checks if deck still has 52 cards, and if not,
def check_new_shoe(deck):
    if len(deck.cards) == 0:
        print("SHUFFLING NEW SHOE!")
        deck = Deck()
        deck.shuffle()
        return deck
    else:
        return deck
    
def play_again():
    valid_answers = ("Y", "N")
    invalid_answer = True
    
    while invalid_answer:
        answer = input(f"Play another hand {valid_answers}? ")
        if answer.upper() in valid_answers:
            invalid_answer = False
            return answer == "Y"
        else:
            print("Sorry, please enter a correct input.")

def display_board(bet, player, dealer):
    clear_output(wait=False)
    print(f"Current bet is {bet}.")
    print('---')
    print(f"Dealer Total: {dealer.total}")
    if len(dealer.face_down_card) == 1:
        print("*** of ***")
    for card in dealer.cards:
        print(f'{card.rank} of {card.suit}')
    print("---")
    print(f"Player Total: {player.total}")          
    for card in player.cards:
        print(f'{card.rank} of {card.suit}')
    print("---")
    
def hand_action():
    valid_answers = ("Hit", "Stay")
    invalid_answer = True
    
    while invalid_answer:
        answer = input(f"What would you like to do? {valid_answers} ")
        if answer in valid_answers:
            invalid_answer = False
            return answer
        else:
            print("Please select a valid action")

def reset_hand(player,dealer):
    player.remove_hand()
    dealer.remove_hand()
    return False

def check_win(bet, player, dealer):
    if player.total > 21:
        print(f"Bust! Dealer Wins! You lost {bet}")
    elif dealer.total > 21 or player.total > dealer.total:
        winnings = bet*2
        player.collect_bet(winnings)
        print(f"Player Wins! Collecting {winnings}")
    elif dealer.total > player.total:
        print(f"Dealer Wins! You lost {bet}")
    else:
        print(f"Push! Collecting {bet}.")

In [ ]:
'''
GAME LOGIC
'''
# Global Varialbes Players
game_on = True
player = Player()
dealer = Dealer()
hands_played = 1

while game_on: 
    # Game Variables
    active_hand = True
    
    # Create & Shuffle Deck
    deck = Deck()
    deck.shuffle()
    
    while active_hand:
        clear_output(wait=False)
        player_action = True
        dealer_action = True
        print(f"Player Balance is {player.balance}. Hand {hands_played}")
        
        #Place Bet
        current_bet = player.place_bet()

        # Deal Cards
        for num in range(2):
            # Deal to Dealer
            deck = check_new_shoe(deck)
            dealer_card = deck.deal_one()

            if num == 0:
                dealer.add_face_down_card(dealer_card)
            else:
                dealer.add_cards(dealer_card)

            # Deal to Player
            deck = check_new_shoe(deck)
            player_card = deck.deal_one()
            player.add_cards(player_card)
        
        # Display hand & ask for action
        display_board(current_bet, player, dealer)
        
        # Player Action
        while player_action:
            
            answer = hand_action()

            if answer == "Hit":
                deck = check_new_shoe(deck)
                player_card = deck.deal_one()
                player.add_cards(player_card)
                display_board(current_bet, player, dealer)
                if player.total > 21:
                    player_action = False
            else:
                player_action = False
        
        # Dealer Action
        dealer.flip_card()
        display_board(current_bet, player, dealer)
        
        if player.total > 21 or dealer.total > 21:
            dealer_action = False
            
        while dealer_action:
            time.sleep(1)
            if dealer.total > 21:
                dealer_action = False
            elif dealer.total < 17:
                deck = check_new_shoe(deck)
                dealer_card = deck.deal_one()
                dealer.add_cards(dealer_card)
            elif 17 <= dealer.total <= 21: 
                dealer_action = False
            
            display_board(current_bet, player, dealer)
                
        # Check who won & reset hand
        check_win(current_bet, player, dealer)
        active_hand = reset_hand(player, dealer)
        
    # When hand is over, ask if player would like to resume
    if player.balance == 0:
        print("You have nothing left to bet! GAME OVER")
        game_on = False
    else:
        hands_played = hands_played + 1
        game_on = play_again()

Current bet is 10.
---
Dealer Total: 24
Three of Clubs
Jack of Diamonds
Ace of Spades
Ten of Clubs
---
Player Total: 14
Four of Spades
King of Clubs
---
Player Wins! Collecting 20


1